### 도쿄 부동산 사건 데이터 수집 
출처 :  <a href = 'http://www.oshimaland.co.jp'> 오시마랜드 </a>

사이트 정보 : 본 웹사이트는 자살사건, 시체유기사건, 살인사건 등 과거에 불미스러운 사건이 있었던 건물을 모아서 보여줍니다.


In [2]:
import pandas as pd
import numpy as np 

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import time

In [3]:
# !pip install googletrans==4.0.0-rc1
# !pip install webdriver-manager
# !pip install requests

## 에어비앤비 주요 지역 탐색 

#### 1. 영어로된 일본 지역 리스트 본어로 전처리 
- 오시마랜드는 영오도 지원되나 일본어로 검색하는게 정확도가 높다 판단하여 일본어로 변경해준다 
- googletrans를 이용해 번역할 경우 정확도가 떨어진다
- 45개 지역 하드코딩으로 번역

In [4]:
from googletrans import Translator

main_district = pd.read_csv('../jieun/main_district_df2.csv')
# district_list  = main_district['neighbourhood_cleansed'].values
# translator = Translator()
# jp_district_list = list(map(lambda district: translator.translate(district, src='en', dest='ja').text.replace(' ',''),district_list))
# jp_district_list[: 10]


In [5]:
main_district

,neighbourhood_cleansed,count
0,Shinjuku Ku,2278
1,Taito Ku,1597
2,Sumida Ku,1290
3,Toshima Ku,1002
4,Shibuya Ku,660
5,Minato Ku,409
6,Setagaya Ku,394
7,Ota Ku,360
8,Nakano Ku,316
9,Chuo Ku,290


#### Inside AIrbne 주요 지역

In [6]:
jp_district_list = [
    '新宿区', '台東区', '墨田区', '豊島区', '渋谷区',
    '港区', '世田谷区', '大田区', '中野区', '中央区',
    '北区', '葛飾区', '杉並区', '江東区', '江戸川区',
    '文京区', '荒川区', '板橋区', '品川区', '千代田区',
    '足立区', '目黒区', '練馬区', '府中市', '八王子市',
    '日野市', '国分寺市', '町田市', '青梅市', '調布市',
    '武蔵野市', 'あきる野市', '三鷹市', '小金井市', '東村山市',
    '多摩市', '国立市', '狛江市', '西東京市', '立川市',
    '小平市', '羽村市', '武蔵村山市', '奥多摩町', '昭島市',
    '福生市'
]

#### 2. geojosn 파일 이용
InsideAirbnb 제공 geo 파일 사용

In [7]:
import json
import pandas as pd

In [8]:
with open('../../Tokyo_Airbnb/data/neighbourhoods.geojson') as f:
    json_f = json.loads(f.read()) # json 라이브러리 이용

geo_df = pd.DataFrame(json_f)
geo_df = pd.read_json('../../Tokyo_Airbnb/data/neighbourhoods.geojson') # pd.read_json 이용


In [9]:
geojson_path ='../../Tokyo_Airbnb/data/neighbourhoods.geojson'
# gdf = gpd.read_file(geojson_path)
# gdf

with open(geojson_path) as f:
    geo_data = json.load(f)

geo_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[139.857803, 35.635799],
       [139.855301, 35.636868],
       [139.850296, 35.637665],
       [139.849899, 35.639267],
       [139.850494, 35.639999],
       [139.858398, 35.638134],
       [139.857895, 35.635864],
       [139.857803, 35.635799]]],
     [[[139.889999, 35.750801],
       [139.890396, 35.750401],
       [139.899796, 35.743732],
       [139.900101, 35.7356],
       [139.897095, 35.726665],
       [139.901398, 35.721466],
       [139.912109, 35.711266],
       [139.916107, 35.707733],
       [139.918701, 35.6978],
       [139.905899, 35.683731],
       [139.889099, 35.680599],
       [139.885803, 35.6768],
       [139.885895, 35.671131],
       [139.886597, 35.669868],
       [139.886993, 35.653866],
       [139.882095, 35.644001],
       [139.875504, 35.640533],
       [139.872894, 35.638535],
       [139.871796, 35.639332],
       [139.863495, 3

In [10]:
features = geo_data['features']

geo_dict = {}
for feature in features:
    coordinates = feature["geometry"]['coordinates']
    neighbourhood = feature["properties"]['neighbourhood']
    if neighbourhood in list(main_district['neighbourhood_cleansed'].values):
          lat_lng = coordinates[0][len(coordinates[0])//2]
          geo_dict[neighbourhood] = lat_lng

geo_dict

{'Edogawa Ku': [[139.857803, 35.635799],
  [139.855301, 35.636868],
  [139.850296, 35.637665],
  [139.849899, 35.639267],
  [139.850494, 35.639999],
  [139.858398, 35.638134],
  [139.857895, 35.635864],
  [139.857803, 35.635799]],
 'Katsushika Ku': [[139.846802, 35.796532],
  [139.8564, 35.790932],
  [139.858795, 35.794464],
  [139.862701, 35.794731],
  [139.871796, 35.797066],
  [139.877396, 35.7938],
  [139.871796, 35.788334],
  [139.871796, 35.784466],
  [139.876694, 35.780865],
  [139.881302, 35.782066],
  [139.889099, 35.783733],
  [139.893799, 35.782734],
  [139.893295, 35.778198],
  [139.880493, 35.771198],
  [139.880798, 35.764732],
  [139.887802, 35.753399],
  [139.888, 35.753201],
  [139.889999, 35.750801],
  [139.880493, 35.748867],
  [139.884903, 35.746601],
  [139.879807, 35.7416],
  [139.880005, 35.7402],
  [139.877594, 35.741135],
  [139.876495, 35.737064],
  [139.874802, 35.7374],
  [139.873703, 35.736866],
  [139.871796, 35.729797],
  [139.871109, 35.729065],
  [139.87

In [11]:
dict_list = list(geo_dict.keys())[:8]
new_dict = {key: value for key, value in geo_dict.items() if key not in dict_list}
new_dict

{'Fussa Shi': [[139.342697, 35.749866],
  [139.3423, 35.746532],
  [139.344894, 35.746864],
  [139.3526, 35.741665],
  [139.350006, 35.747799],
  [139.354599, 35.745602],
  [139.356003, 35.745068],
  [139.354904, 35.742802],
  [139.359802, 35.739666],
  [139.358994, 35.7374],
  [139.352905, 35.736065],
  [139.354202, 35.728001],
  [139.355301, 35.725201],
  [139.353104, 35.724533],
  [139.351501, 35.722267],
  [139.350296, 35.722332],
  [139.343796, 35.721066],
  [139.341797, 35.721268],
  [139.338806, 35.718601],
  [139.337601, 35.715199],
  [139.332596, 35.712799],
  [139.328796, 35.7118],
  [139.328201, 35.729134],
  [139.320908, 35.733067],
  [139.313797, 35.747131],
  [139.318802, 35.750866],
  [139.320999, 35.750134],
  [139.322296, 35.753201],
  [139.324905, 35.754665],
  [139.327103, 35.753201],
  [139.328201, 35.753201],
  [139.329208, 35.753201],
  [139.329498, 35.752335],
  [139.331604, 35.752998],
  [139.334793, 35.750732],
  [139.3367, 35.751801],
  [139.340805, 35.749733]

## 크롤링 
- 크롤링 메서드 생성 

####  1. oshimaland 웹사이트 불러오기


In [13]:
def loadWeb(driver, url):
    # header 추가하기 

    
    # 화면 최대 크기 설정 
    driver.get(url)
    driver.maximize_window()
    #모든 창이 로딩 될떄까지 기다려 줌 
    time.sleep(3)
    #팝업창 화면 전환 후 닫아주기
    multi_window = driver.window_handles
    if(len(multi_window)>1):
        driver.switch_to.window(driver.window_handles[-1])
        driver.close()
        time.sleep(3)
        # 접근 페이지 다시 요청
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(url)
    
    # 전체 지도 축소 
    # driver.execute_script("document.body.style.zoom='60%'")    
    # time.sleep(2)




#### 2. 부동산 관련 사건 정보 수집

In [14]:
from selenium.webdriver.common.action_chains import ActionChains

from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException, MoveTargetOutOfBoundsException, StaleElementReferenceException, WebDriverException

def get_fire(driver, tokyo_data_dict, district):
        # fire 객체 수집 
        fire_object_list = driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > div > div.mk-annotation-container')
        fire_list = fire_object_list.find_elements(By.CSS_SELECTOR, '.map-fire')
        print('fire 개수: ', len(fire_list))
        for fire in fire_list:
            try: 
                is_enabled = fire.is_enabled()
                actions = ActionChains(driver)                 
                if is_enabled:
                    # 클릭이 가능하며 현재 화면에 표시 될 때 클릭 수행
                    # print('click OK')
                    if fire.is_displayed():
                        # print('display OK')
                        # x_offset = 200  # x 좌표를 조정할 값
                        # y_offset = 200  # y 좌표를 조정할 값
                        # driver.execute_script("arguments[0].scrollIntoView();", fire)
                        # actions.move_to_element_with_offset(fire, x_offset, y_offset)
                        # actions.click()
                        # actions.perform()

                        # 팝업창 클릭                         
                        actions = ActionChains(driver)                 
                        actions.move_to_element(fire).click().perform()

                        # 팝업 정보 가져오기 
                        # time.sleep(5)
                        WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.property-info-date > span')))
                        info_date = driver.find_element(By.CSS_SELECTOR, 'li.property-info-date > span').text
                        info_address = driver.find_element(By.CSS_SELECTOR, 'li.property-info-address').text
                        info_content = driver.find_element(By.CSS_SELECTOR, 'div.popup-property-info > section > ul > li:nth-child(3)').text

                        tokyo_data_dict.append({
                            'district': district,
                            'date': info_date,
                            'address': info_address,
                            'content': info_content
                        })

                        time.sleep(3)

                    else:
                        pass
                        # print('display NOT')
                else:
                     pass
                    # print('click not')

            except (NoSuchElementException, NoSuchWindowException, MoveTargetOutOfBoundsException, StaleElementReferenceException, WebDriverException) as e:
                # 에러 처리
                print(f"Error occurred: {e}")

                # 필요한 경우 WebDriver 인스턴스 재초기화 등의 작업 수행
                # driver = initialize_driver()

                continue


#### 3. 지역 검색 

#### 3-1 도쿄 주요 지역 검색 

In [15]:
from tqdm import tqdm, tqdm_notebook

tokyo_data_dict = []

def search_fire_distrinct(driver, jp_district_list):
    # 주요 지역 검색 
    for district in tqdm(jp_district_list):
        print(district)
        region_search_tab = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
        # 검색어 clear
        region_search_tab.clear()
        # 지역 검색
        region_search_tab.send_keys(district)
        time.sleep(2)
        # 검색 버튼 클릭 
        search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
        search_btn.send_keys(Keys.ENTER)
        # ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (751, 3641)
        time.sleep(8)
        # fire 객체 정보 수집 
        get_fire(driver,tokyo_data_dict, district)




#### 3-2 도쿄 지역 위도 경도 검색

In [37]:
def search_fire_coordinate(driver, geo_dict):
    # 주요 지역 위도 경도 검색 
    for key, value in tqdm(geo_dict.items()):
       
        for lat_lng in value:
            print(key)
            region_search_tab = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
            region_search_tab.clear()  # 검색어 clear
            lng = lat_lng[1]
            lat = lat_lng[0]
            #경도 위도 검색
            search  = f'{lng} {lat}'
            region_search_tab.send_keys(search)
            time.sleep(2)
            # 검색 버튼 클릭 
            search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
            search_btn.send_keys(Keys.ENTER)
            # ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (751, 3641)
            time.sleep(8)
            # fire 객체 정보 수집 
            get_fire(driver,tokyo_data_dict, key)



#### 병렬 처리

In [16]:
def search_and_collect_fire_info(driver, lat, lng, district):
    region_search_tab = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
    region_search_tab.clear()
    search = f'{lng} {lat}'
    region_search_tab.send_keys(search)
    time.sleep(2)
    search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
    search_btn.send_keys(Keys.ENTER)
    time.sleep(8)
    tokyo_data_dict = []
    get_fire(driver, tokyo_data_dict, district)
    return tokyo_data_dict



In [24]:
from concurrent.futures import ThreadPoolExecutor, as_completed

tokyo_data_dict = []

def search_fire_coordinate_with_thread(driver, geo_dict):

    with ThreadPoolExecutor() as executor:
        futures = []
        for district, coordinates in geo_dict.items():
            for lat, lng in coordinates:
                future = executor.submit(search_and_collect_fire_info, driver, lat, lng, district)
                futures.append(future)

        try: # NameError: name 'concurrent' is not defined
            for future in tqdm(as_completed(futures), total=len(futures)):
                try:
                    result = future.result()
                    if result is not None:
                        tokyo_data_dict.extend(result)
                except Exception as e:
                    print(f"Error occurred while processing a thread: {e}")
        except KeyboardInterrupt:
            # If the execution is interrupted (e.g., user presses Ctrl+C), cancel remaining tasks.
            for future in futures:
                future.cancel()
        

### 크롤링 

In [48]:

dict_list = list(geo_dict.keys())[36:]
rest_dict = {key: value for key, value in geo_dict.items() if key  in dict_list}
list(rest_dict.keys())

['Kita Ku',
 'Tachikawa Shi',
 'Hachioji Shi',
 'Mitaka Shi',
 'Okutama Machi',
 'Musashino Shi',
 'Musashimurayama Shi',
 'Ome Shi',
 'Akiruno Shi',
 'Hamura Shi']

In [49]:
if __name__ == "__main__":
    # 드라이버 생성 및 웹 로딩
    driver = webdriver.Firefox()
    url = 'https://www.oshimaland.co.jp/'
    loadWeb(driver, url)


    # 병렬 처리
    # 화재 데이터 수집 
    # tokyo_data_dict = search_fire_coordinatewithThread(driver, test_dict)

    # 기존 크롤링
    # 주요 지역명 검색 
    # search_fire(driver, jp_district_list) 
    # 위도 경도 검색 
    # 지도를 60% 정도로 축소 뒤 수집 
    search_fire_coordinate(driver, rest_dict)


driver.quit()

  0%|          | 0/10 [00:00<?, ?it/s]

Kita Ku
fire 개수:  11
Error occurred: Message: The element with the reference e1d990e2-11a3-4293-829a-96d7863574e9 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 67689b33-d94d-4d89-ab8c-c4330143706a is stale; either its node document is not the active document, or it is no longer connected to the DOM

Kita Ku
fire 개수:  11
Kita Ku
fire 개수:  12
Kita Ku
fire 개수:  4
Kita Ku
fire 개수:  23
Error occurred: Message: The element with the reference 56b95bff-fc5f-4b0b-9038-dfe3f336fb76 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference eb7842da-3ea6-4087-b0d4-0e46682ff8dd is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 1e4f1de3-df46-4abc-9871-df55e5d5e2f4 is stale; eithe

 10%|█         | 1/10 [31:54<4:47:09, 1914.40s/it]

Tachikawa Shi
fire 개수:  8
Error occurred: Message: The element with the reference af356921-ae10-41aa-87b2-f0e90d0735b3 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 2be12e3b-f93a-4d67-835b-35edcae32af0 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Tachikawa Shi
fire 개수:  19
Tachikawa Shi
fire 개수:  19
Error occurred: Message: The element with the reference fe2eeb03-f302-4605-acea-ecd2609c9b57 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 3857f3e1-ebf1-4e1a-9733-623c4b2a3e16 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference ac15d75d-3781-48bd-a90a-960086dd75d0 is stale; either its node document is n

 20%|██        | 2/10 [1:01:06<4:02:33, 1819.14s/it]

...

fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  1
Akiruno Shi
fire 개수:  1
Akiruno Shi
fire 개수:  1
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  1
Akiruno Shi
fire 개수:  1
Akiruno Shi
fire 개수:  1
Akiruno Shi
fire 개수:  1
Akiruno Shi
fire 개수:  1
Akiruno Shi
fire 개수:  1
Akiruno Shi
fire 개수:  1
Akiruno Shi
fire 개수:  2
Akiruno Shi
fire 개수:  3
Akiruno Shi
fire 개수:  2
Akiruno Shi
fire 개수:  2
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akiruno Shi
fire 개수:  0
Akir

 90%|█████████ | 9/10 [3:29:00<19:29, 1169.49s/it]

fire 개수:  0
Hamura Shi
fire 개수:  0
Hamura Shi
fire 개수:  6
Hamura Shi
fire 개수:  11
Hamura Shi
fire 개수:  14
Error occurred: Message: The element with the reference 7cabe038-e3b2-40fc-8cb8-2470127f5e91 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference f740cce6-2322-4cff-b19c-bf4e14b29fd6 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference a6fec13b-c241-4ebc-a8bc-5d3bbc6466ac is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 28c6b8f2-0afb-4139-9d3a-2d0f658698e8 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 02c2daee-6520-4bad-9ecd-033afe3109d2 is stale; eit

100%|██████████| 10/10 [3:40:05<00:00, 1320.51s/it]

fire 개수:  0


#### 발생 에러들

1. WebDriverException: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"No node with given id found"} : Chrome driver 문제
2. StaleElementReferenceException: Message: The element with the reference b3212e1d-3be9-4d7a-9bd2-9f736fc42e4a is stale; either its node document is not the active document, or it is no longer connected to the DOM
3. AttributeError: 'NoneType' object has no attribute 'text'
4. MoveTargetOutOfBoundsException

### 크롤링 데이터 저장


In [50]:
tokyo_data_dict

[{'district': 'Akishima Shi',
  'date': '不明',
  'address': '東京都立川市西砂町二丁目38-5 レオパレス西砂川 207号室',
  'content': '告知事項が御座います'},
 {'district': 'Akishima Shi',
  'date': '不明',
  'address': '東京都立川市西砂町二丁目38-5 レオパレス西砂川 207号室',
  'content': '告知事項が御座います'},
 {'district': 'Akishima Shi',
  'date': '令和4年5月21日',
  'address': '東京都立川市西砂町一丁目15-25 クレイノソレイユ立川',
  'content': '102号室\n告知事項有'},
 {'district': 'Akishima Shi',
  'date': '令和4年5月21日',
  'address': '東京都立川市西砂町一丁目15-25 クレイノソレイユ立川',
  'content': '102号室\n告知事項有'},
 {'district': 'Akishima Shi',
  'date': '令和4年5月21日',
  'address': '東京都立川市西砂町一丁目15-25 クレイノソレイユ立川',
  'content': '102号室\n告知事項有'},
 {'district': 'Akishima Shi',
  'date': '令和4年5月21日',
  'address': '東京都立川市西砂町一丁目15-25 クレイノソレイユ立川',
  'content': '102号室\n告知事項有'},
 {'district': 'Akishima Shi',
  'date': '2021年',
  'address': '東京都昭島市武蔵野二丁目22-16',
  'content': '独身寮。変死見つかる。'},
 {'district': 'Akishima Shi',
  'date': '不明',
  'address': '東京都立川市一番町1丁目14-1 ハイホーム立川一番町 5階/512号室',
  'content': '告知事項有り\n\nリフォーム済み（2

In [51]:
tokyo_df = pd.DataFrame(tokyo_data_dict)
tokyo_df

,district,date,address,content
0,Akishima Shi,不明,東京都立川市西砂町二丁目38-5 レオパレス西砂川 207号室,告知事項が御座います
1,Akishima Shi,不明,東京都立川市西砂町二丁目38-5 レオパレス西砂川 207号室,告知事項が御座います
2,Akishima Shi,令和4年5月21日,東京都立川市西砂町一丁目15-25 クレイノソレイユ立川,102号室\n告知事項有
3,Akishima Shi,令和4年5月21日,東京都立川市西砂町一丁目15-25 クレイノソレイユ立川,102号室\n告知事項有
4,Akishima Shi,令和4年5月21日,東京都立川市西砂町一丁目15-25 クレイノソレイユ立川,102号室\n告知事項有
...,...,...,...,...
2839,Hamura Shi,2003〜2012年,東京都羽村市川崎二丁目1-1レオパレスオリバ川崎201,不明\n※現在空室なし
2840,Hamura Shi,２０００年頃の春〜夏,東京都福生市加美平一丁目28-9,この会社の隣にある八百屋のおじさんが自家用車をこちらの会社の駐車場に持ち込み車内で自殺しまし...
2841,Hamura Shi,2010,東京都羽村市川崎二丁目6-19,マンションから飛び降り自殺
2842,Hamura Shi,令和4年10月6日,東京都福生市福生1688 コーポラス天神3,2階 角部屋\n告知事項あり


In [52]:
# 결측치 제거 
df_cleaned = tokyo_df.dropna(how='any', axis=0)

for column in df_cleaned.columns:
    df_cleaned = df_cleaned[df_cleaned[column] != '']

# 중복데이터 제거 
df_cleaned = df_cleaned.drop_duplicates()
df_cleaned

,district,date,address,content
0,Akishima Shi,不明,東京都立川市西砂町二丁目38-5 レオパレス西砂川 207号室,告知事項が御座います
2,Akishima Shi,令和4年5月21日,東京都立川市西砂町一丁目15-25 クレイノソレイユ立川,102号室\n告知事項有
6,Akishima Shi,2021年,東京都昭島市武蔵野二丁目22-16,独身寮。変死見つかる。
7,Akishima Shi,不明,東京都立川市一番町1丁目14-1 ハイホーム立川一番町 5階/512号室,告知事項有り\n\nリフォーム済み（2017年07月）
13,Akishima Shi,令和4年3月1日,東京都立川市上砂町一丁目9-19,ニューノース壱番館306号室\n変死体発見
...,...,...,...,...
2815,Hamura Shi,2003〜2012年,東京都羽村市川崎二丁目1-1レオパレスオリバ川崎201,不明\n※現在空室なし
2816,Hamura Shi,２０００年頃の春〜夏,東京都福生市加美平一丁目28-9,この会社の隣にある八百屋のおじさんが自家用車をこちらの会社の駐車場に持ち込み車内で自殺しまし...
2841,Hamura Shi,2010,東京都羽村市川崎二丁目6-19,マンションから飛び降り自殺
2842,Hamura Shi,令和4年10月6日,東京都福生市福生1688 コーポラス天神3,2階 角部屋\n告知事項あり


In [53]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 2843
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   district  1309 non-null   object
 1   date      1309 non-null   object
 2   address   1309 non-null   object
 3   content   1309 non-null   object
dtypes: object(4)
memory usage: 51.1+ KB


In [55]:
# tokyo_df.to_excel('selenium_tokyo_data_geo.xlsx')
df_cleaned.to_csv('selenium_tokyo_data_geo(-Hamura Shi).csv')

In [56]:
df1 = pd.read_csv('selenium_tokyo_data_geo.csv', index_col=0)
df2 = pd.read_csv('selenium_tokyo_data_geo(-Fuchu Shi).csv', index_col=0)
df3 = pd.read_csv('selenium_tokyo_data_geo(-Hamura Shi).csv', index_col=0)

all_data = pd.concat([df1, df2, df3], ignore_index=True)
all_data

,district,date,address,content
0,Katsushika Ku,24年,東京都葛飾区西水元四丁目8-13,老婆がお風呂場で死亡
1,Katsushika Ku,平成30年2月14日,埼玉県八潮市大瀬1654,火災による死亡
2,Katsushika Ku,平成24年1月9日,埼玉県三郷市高州二丁目21-1,火災による死亡
3,Katsushika Ku,令和3年11月21日,埼玉県三郷市高州一丁目348-1,火災による死亡
4,Katsushika Ku,2023年,埼玉県三郷市鷹野一丁目255,部屋番号は記載しないが自殺あり
...,...,...,...,...
2407,Hamura Shi,2003〜2012年,東京都羽村市川崎二丁目1-1レオパレスオリバ川崎201,不明\n※現在空室なし
2408,Hamura Shi,２０００年頃の春〜夏,東京都福生市加美平一丁目28-9,この会社の隣にある八百屋のおじさんが自家用車をこちらの会社の駐車場に持ち込み車内で自殺しまし...
2409,Hamura Shi,2010,東京都羽村市川崎二丁目6-19,マンションから飛び降り自殺
2410,Hamura Shi,令和4年10月6日,東京都福生市福生1688 コーポラス天神3,2階 角部屋\n告知事項あり


#### 기존 오시마 데이터랑 중복 확인 

In [57]:
oshima_yun = pd.read_csv('../../Oshimaland_data/yunyoung/selenium_tokyo_data.csv', index_col=0)
oshima_ji = pd.read_csv('../../Oshimaland_data/jieun/oshimaland_dataset_final.csv')


In [58]:
# 결측치 제거 
oshima_yun = oshima_yun.dropna(how='any', axis=0)

for column in oshima_yun.columns:
    oshima_yun = oshima_yun[oshima_yun[column] != '']


# # 중복데이터 제거 
oshima_yun = oshima_yun.drop_duplicates()
oshima_yun = oshima_yun.reindex(columns=['district', 'address', 'content', 'date'])
oshima_yun.rename(columns={'content': 'info'}, inplace=True)
oshima_yun

,district,address,info,date
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日
2,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年
4,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日
6,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日
8,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日
...,...,...,...,...
298,羽村市,東京都羽村市緑ヶ丘五丁目1-50,駐車スペース横の部屋で住民男性が首吊り自殺,2005
299,武蔵村山市,東京都羽村市羽中一丁目3-37,告知事項,不明
300,昭島市,東京都武蔵村山市中央一丁目24-3ハウスオブブーケA 1階,死体発見,平成24年
301,福生市,東京都昭島市田中町一丁目34-9,火災による死亡,令和2年9月7日


In [59]:
oshima_ji_rename = oshima_ji.reindex(columns=[ 'address', 'info', 'occurred_date'])
oshima_ji_rename.rename(columns={'occurred_date': 'date'}, inplace=True)
oshima_ji_rename = oshima_ji_rename.drop_duplicates()
oshima_ji_rename.reset_index(drop=True, inplace=True)
oshima_ji_rename.dropna(inplace=True)
oshima_ji_rename = oshima_ji_rename.reindex(columns = ['district', 'address', 'info', 'date'])
oshima_ji_rename['district'] = oshima_ji_rename['address'].apply(lambda x: next((district for district in jp_district_list if district in x), x))
oshima_ji_rename.reindex(columns=['district', 'address', 'info', 'occurred_date'])

oshima_ji_rename


,district,address,info,date
0,新宿区,東京都新宿区歌舞伎町一丁目202,819号室で女性死亡,平成30年5月23日
1,新宿区,東京都新宿区歌舞伎町一丁目202アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月18日
2,新宿区,東京都新宿区西新宿七丁目1813,告知事項あり\r\n※管理会社に直接確認済み\r\n\r\nハイム大成ビル 403号室\r\...,1年以内
3,新宿区,東京都新宿区西新宿七丁目1713パレステュディオ新宿WEST,１４Ｆから女性が飛び降り自殺,令和元年9月23日
4,新宿区,東京都新宿区西新宿七丁目2231,事故死,2016
...,...,...,...,...
561,武蔵野市,東京都武蔵野市緑町二丁目34都営武蔵野緑町二丁目第2アパート4号棟,【青酸カリによる死者が出たそうです】\r\n4号棟の自治会役員の方の証言で、団地内で青酸カリ...,不明
562,武蔵野市,東京都武蔵野市緑町二丁目64 都営武蔵野緑町2丁目第3アパート4号棟,死体発見,平成17年12月28日
563,武蔵野市,東京都武蔵野市西久保三丁目128 アネシスヴィーゼ205号室,告知事項あり,不明
564,武蔵野市,東京都武蔵野市西久保三丁目1118,不審死,2022年11月


In [60]:
combined_df = pd.concat([oshima_yun, oshima_ji_rename], ignore_index=True)
combined_df = combined_df.drop_duplicates()
combined_df

,district,address,info,date
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日
...,...,...,...,...
803,武蔵野市,東京都武蔵野市緑町二丁目34都営武蔵野緑町二丁目第2アパート4号棟,【青酸カリによる死者が出たそうです】\r\n4号棟の自治会役員の方の証言で、団地内で青酸カリ...,不明
804,武蔵野市,東京都武蔵野市緑町二丁目64 都営武蔵野緑町2丁目第3アパート4号棟,死体発見,平成17年12月28日
805,武蔵野市,東京都武蔵野市西久保三丁目128 アネシスヴィーゼ205号室,告知事項あり,不明
806,武蔵野市,東京都武蔵野市西久保三丁目1118,不審死,2022年11月


In [61]:
new_oshima = all_data.reindex(columns=['district', 'address', 'content', 'date'])
new_oshima.rename(columns={'content': 'info'}, inplace=True)
new_oshima

,district,address,info,date
0,Katsushika Ku,東京都葛飾区西水元四丁目8-13,老婆がお風呂場で死亡,24年
1,Katsushika Ku,埼玉県八潮市大瀬1654,火災による死亡,平成30年2月14日
2,Katsushika Ku,埼玉県三郷市高州二丁目21-1,火災による死亡,平成24年1月9日
3,Katsushika Ku,埼玉県三郷市高州一丁目348-1,火災による死亡,令和3年11月21日
4,Katsushika Ku,埼玉県三郷市鷹野一丁目255,部屋番号は記載しないが自殺あり,2023年
...,...,...,...,...
2407,Hamura Shi,東京都羽村市川崎二丁目1-1レオパレスオリバ川崎201,不明\n※現在空室なし,2003〜2012年
2408,Hamura Shi,東京都福生市加美平一丁目28-9,この会社の隣にある八百屋のおじさんが自家用車をこちらの会社の駐車場に持ち込み車内で自殺しまし...,２０００年頃の春〜夏
2409,Hamura Shi,東京都羽村市川崎二丁目6-19,マンションから飛び降り自殺,2010
2410,Hamura Shi,東京都福生市福生1688 コーポラス天神3,2階 角部屋\n告知事項あり,令和4年10月6日


In [62]:
combined_df2 = pd.concat([combined_df, new_oshima], ignore_index=True)
combined_df2 = combined_df2.drop_duplicates()
combined_df2

,district,address,info,date
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日
...,...,...,...,...
3210,Hamura Shi,東京都羽村市川崎二丁目1-1レオパレスオリバ川崎201,不明\n※現在空室なし,2003〜2012年
3211,Hamura Shi,東京都福生市加美平一丁目28-9,この会社の隣にある八百屋のおじさんが自家用車をこちらの会社の駐車場に持ち込み車内で自殺しまし...,２０００年頃の春〜夏
3212,Hamura Shi,東京都羽村市川崎二丁目6-19,マンションから飛び降り自殺,2010
3213,Hamura Shi,東京都福生市福生1688 コーポラス天神3,2階 角部屋\n告知事項あり,令和4年10月6日


#### 중복 데이터 확인 
- 7개의 지역만 추가 트롤링 했을때 기존 807개의 데이터에서  1183개의 데이터 즉 376 개의 새로운 데이터 수집 
- 대략적으로 46개의 지역을 다 크롤링 할 시 2000개 이상의 데이터 추가 수집 가능 

In [63]:
# address 컬럼의 값이 중복되면 하나만 남기고 제거
fin_data = combined_df2.drop_duplicates(subset=['address'], keep='first')
fin_data

,district,address,info,date
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日
...,...,...,...,...
3208,Hamura Shi,東京都福生市武蔵野台二丁目34-2 レヂオンス武蔵野台 1階,告知事項有\n\nリフォーム完成予定：平成29年11月末,不明
3209,Hamura Shi,東京都福生市加美平四丁目1-11 105号室,退院し仕事復帰の予定だった男性が出勤せず、連絡も取れない。通報後死亡が確認された。,2010年〜2012年
3210,Hamura Shi,東京都羽村市川崎二丁目1-1レオパレスオリバ川崎201,不明\n※現在空室なし,2003〜2012年
3211,Hamura Shi,東京都福生市加美平一丁目28-9,この会社の隣にある八百屋のおじさんが自家用車をこちらの会社の駐車場に持ち込み車内で自殺しまし...,２０００年頃の春〜夏


In [64]:
fin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2286 entries, 0 to 3213
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   district  2286 non-null   object
 1   address   2286 non-null   object
 2   info      2286 non-null   object
 3   date      2286 non-null   object
dtypes: object(4)
memory usage: 89.3+ KB


In [66]:
fin_data.to_csv('oshima_data_combine_fin.csv')

#### 지도 내 줌인 줌아웃 실패 

In [ ]:
# from selenium.webdriver.common.action_chains import ActionChains
# # https://stackoverflow.com/questions/64474653/how-to-webscrape-a-canvas-element-with-python-selenium
# canvas = driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > canvas.rt-root')
# action = ActionChains(driver)

# map_canvas = driver.find_element_by_css_selector("div.mk-map-view > canvas.rt-root")

# from selenium.webdriver.common.action_chains import ActionChains

# # 지도 요소 위에서 마우스를 움직임
# actions = ActionChains(driver)
# actions.move_to_element(map_canvas).click().perform()

# actions.move_by_offset(0, 200).perform() # 줌인만 가능 

# #줌인 줌아웃 반복 
# action.move_to_element(canvas).move_by_offset(0, 0).click_and_hold().perform()
# action.move_by_offset(-200,0).release().perform()

#### Request API 이용한 방법 

In [ ]:
import json
from  bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import urllib.request as req
from tqdm import tqdm, tqdm_notebook
import time
import requests

In [ ]:
#TODO shinjuku ku test
search_region = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
search_region.send_keys(jp_district_list[20])
time.sleep(2)
# 검색 버튼 클릭 
search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
search_btn.send_keys(Keys.ENTER)

time.sleep(2)

In [ ]:
json_url = 'https://api.oshimaland.co.jp/map'
headers = {
    "User-Agent": "Mozilla/5.0",
    "Origin": "https://www.oshimaland.co.jp"
}
data = {
    # 'keys': ["1330021123012133", "1330021123012311", "1330021123013022", "1330021123013200"]
    'keys' : ['1330021123013021',  '1330021123013023',  '1330021123013201']
}

response = requests.post(json_url, headers=headers, json=data)
json_data = response.json()

In [ ]:
from itertools import chain

origin = list(data.values())
flatten_list = list(chain(*origin))
flatten_list

In [ ]:
jsonValueList = []
for idx in range(len(flatten_list)):
    for idx2 in range(len(json_data['markers'][flatten_list[idx]])):
        jsonValueList.append(json_data['markers'][flatten_list[idx]][idx2]['key'])

In [ ]:
getDataList2 = []
for idx in tqdm_notebook(range(len(jsonValueList))):
    json_url = f'https://www.oshimaland.co.jp/d/{jsonValueList[idx]}.json'
    url = req.Request(json_url, headers={"User-Agent": "Mozilla/5.0"})
    code = req.urlopen(url)
    soup = BeautifulSoup(code, 'html.parser')
    json_data = json.loads(soup.text)
    getDataList2.append({
        'info' : json_data['info'],
        'address' : json_data['ad'],
        'dt' : json_data['dt'],
        'cr' : json_data['cr'],
     })